In [1]:
# == importar bibliotecas ==

import pandas as pd 
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import SelectKBest
from xgboost import XGBClassifier
import numpy as np
import joblib
import unicodedata
from transformers import AutoTokenizer  
from transformers import AutoTokenizer  
from transformers import TFBertModel, BertModel
import torch

2023-12-11 21:36:48.945034: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 21:36:49.728044: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-11 21:36:53.745805: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/

In [ ]:
# == ler dataset ==

data = pd.read_parquet('artigos_tratados_bert_lg.parquet')

In [5]:
# == limpar dataset == 

data = data[data['Conteudo'] != '']
data.drop(columns=['Conteudo', 'URL'], inplace=True)
data['Vies'] = data['Vies'].map({'direita':2,
                                'centro': 1,
                                'esquerda': 0})

In [6]:
# == separar X e y ==

X = data.drop(columns=['Partido', 'Vies'])
y = data.Vies

In [7]:
# == split treino e teste == 

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y,
    test_size=0.2,
    random_state=42,
    stratify=data.Partido
)

In [12]:
# == preparar o modelo com melhor desempenho == 

best_params = {
    "selection__k": 800,
    "estimator__gamma": 0,
    "estimator__alpha": 22,
    "estimator__lambda": 0,
    "estimator__colsample_bytree": 1
}
pipeline = Pipeline([
    ('scaling', MaxAbsScaler()), 
    ('selection', SelectKBest()),
    ('ros', RandomOverSampler(random_state=42)),
    ('estimator', XGBClassifier(seed=42))
])
pipeline.set_params(**best_params)
pipeline.fit(X, y)
joblib.dump(pipeline, 'pipeline.pkl')

['pipeline.pkl']

In [13]:
# == dar um load no modelo == 

pipeline = joblib.load('pipeline.pkl')

In [14]:
# == receber input de novo artigo == 

artigo = 'Ao participar da abertura do 26° encontro do Foro de São Paulo, o presidente Luiz Inácio Lula da Silva criticou a “direita fascista”, pediu união da esquerda no Brasil e disse que tem orgulho de ser chamado de “comunista”. Na cerimônia, ocorrida nesta quinta-feira, 29, em Brasília, o petista se reuniu com os partidos de esquerda da América Latina. Em seu discurso, o petista admitiu que o “discurso do patriotismo e da familia” não estão alinhados com o progressismo. “Aqui no Brasil, nós enfrentamos o discurso do costume, o discurso da família, o discurso do patriotismo. Ou seja, aqui nós enfrentamos o discurso que a gente aprendeu a historicamente combater”, afirmou o presidente da República. “Eles nos acusam de comunistas, achando que nós ficamos ofendidos com isso. Nós não ficamos ofendidos. Ficaríamos ofendidos se nos chamassem de nazista, de neofascista, de terrorista. Mas de comunista, de socialista, nunca. Isso não nos ofende. Isso nos orgulha muitas vezes”, continuou o petista. Fundado em 1990, por iniciativa do Partido dos Trabalhadores (PT) do Brasil e do Partido Comunista de Cuba, o Foro de São Paulo é um encontro dos partidos de esquerda da América Latina e tem como lema neste ano “Integração Regional para Avançar a Soberania Latino-americana e Caribenha”. Após a pandemia, o grupo retoma suas reuniões presenciais anuais. A última edição ocorreu em Caracas , capital da Venezuela, em 2019. Além de Lula, estiverem presentes a presidente do PT, Gleisi Hoffmann, a ministra da Ciência e Tecnologia e presidente do PCdoB, Luciana Santos, e o presidente da Embratur, Marcelo Freixo. Manifestantes contrários ao encontro protestaram na porta da sede do evento. Além disso, partidos de direita, como o PL, repudiaram a presença do que chamam de “ditadores”. No Twitter, a oposição levantou a hashtag #ForaForodeSP.'
artigo

'Ao participar da abertura do 26° encontro do Foro de São Paulo, o presidente Luiz Inácio Lula da Silva criticou a “direita fascista”, pediu união da esquerda no Brasil e disse que tem orgulho de ser chamado de “comunista”. Na cerimônia, ocorrida nesta quinta-feira, 29, em Brasília, o petista se reuniu com os partidos de esquerda da América Latina. Em seu discurso, o petista admitiu que o “discurso do patriotismo e da familia” não estão alinhados com o progressismo. “Aqui no Brasil, nós enfrentamos o discurso do costume, o discurso da família, o discurso do patriotismo. Ou seja, aqui nós enfrentamos o discurso que a gente aprendeu a historicamente combater”, afirmou o presidente da República. “Eles nos acusam de comunistas, achando que nós ficamos ofendidos com isso. Nós não ficamos ofendidos. Ficaríamos ofendidos se nos chamassem de nazista, de neofascista, de terrorista. Mas de comunista, de socialista, nunca. Isso não nos ofende. Isso nos orgulha muitas vezes”, continuou o petista. 

In [15]:
# == aplicar tratamentos == 

artigo = artigo.lower()
artigo = ''.join(artigo.split('.')[:-2])

partidos = [
    'novo', 'pcb', 'pdt', 
    'pl', 'pp', 'psol', 'pstu', 
    'mdb', 'pcdob', 'psb', 'pt', 
    'pv', 'rede', 'união brasil'
]
for partido in partidos:
    artigo = artigo.replace(f' {partido} ', ' ')
    
artigo = ' '.join(word for word in artigo.split(' ') if word.isalnum())
artigo = unicodedata.normalize('NFKD', artigo)
artigo = artigo.encode('ASCII', 'ignore')
artigo = artigo.decode('utf-8')
artigo = artigo.replace('[^a-zA-Z]', ' ')
for n in range(30):
    artigo = artigo.replace('  ', ' ')

In [ ]:
# == funções auxiliares de embedding do artigo == 

def bert_text_preparation(text, tokenizer):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text, truncation=True, max_length=512, padding = True)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2][1:]

    token_embeddings = hidden_states[-1]
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings

# tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False)
# joblib.dump(tokenizer, 'tokenizer.pkl')
# model = BertModel.from_pretrained('neuralmind/bert-large-portuguese-cased', output_hidden_states = True)
# joblib.dump(model, 'bert_model.pkl')

In [17]:
# == dar load no modelo bert e tokenizer == 

tokenizer = joblib.load('tokenizer.pkl')
model = joblib.load('bert_model.pkl')

In [18]:
# == obter dataframe de embedding do artigo == 

tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(artigo, tokenizer)
list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
bert_emb = np.array(list_token_embeddings).mean(axis=0)

input_model = pd.DataFrame(
    data=bert_emb, 
    index=[f"emb_{i + 1}" for i in range(len(bert_emb))]
).T

In [19]:
# == gerar predict do artigo == 

pred = pipeline.predict(input_model)[0]

In [20]:
# == classificar == 

if pred == 0:
    pred = 'Esquerda'
elif pred == 1:
    pred = 'Centro'
else:
    pred = 'Direita'

In [21]:
# == gerar probabilidade de predict == 

predict_proba = pipeline.predict_proba(input_model)[0]

In [22]:
# == montar dicionário de probabilidade == 

prob_df = pd.DataFrame({
    'Esquerda': predict_proba[0],
    'Centro': predict_proba[1],
    'Direita': predict_proba[2]
}, index=[1]).T

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    go.Bar(
        x=prob_df[1],
        y=prob_df.index,
        orientation='h',
        marker=dict(color = 'orange'),
        marker_line=dict(width=1, color='#6e2f0b')
    )
)

fig.update_layout(
    xaxis_title="Probabilidade", 
    yaxis_title="Viés",
    yaxis=dict(tickfont=dict(color="#6e2f0b")),
    xaxis=dict(tickfont=dict(color="#6e2f0b")),
    xaxis_range = [0,1],
    width=1000,
    height=450
)
fig.update_xaxes(
    showline=True, 
    linewidth=2, 
    linecolor='#6e2f0b', 
    gridcolor='#F67C41', 
    tickcolor='#6e2f0b', 
    tickfont=dict(size=16),
    title=dict(font=dict(size=16)),
    title_font_color="#6e2f0b", 
    color="#6e2f0b"
)
fig.update_yaxes(
    showline=True, 
    linewidth=2, 
    linecolor='#6e2f0b', 
    gridcolor='#F67C41', 
    tickcolor='#6e2f0b', 
    tickfont=dict(size=16),
    title=dict(font=dict(size=16)),
    title_font_color="#6e2f0b", 
    color="#6e2f0b"
)
    
#fig.update(layout_xaxis_range = [0,1])
fig.show()

In [23]:
prob_df

,1
Esquerda,0.653328
Centro,0.341703
Direita,0.004969
